In [1]:
from finlab.crawler import table_date_range, update_table, to_pickle, out, commit,merge, date_range
import datetime
from io import StringIO
import requests
from bs4 import BeautifulSoup
import time

import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
#import pandas_datareader.data as web

import os
from openpyxl import load_workbook

def MarginTrading(date):
    URL = 'https://www.twse.com.tw/exchangeReport/MI_MARGN?response=csv&date='+date.strftime("%Y%m%d")+'&selectType=ALL'
    res = requests.get(URL)
    
    if len(res.text) < 1000:
        print('holiday')
        return pd.DataFrame()

   
    df1 = pd.read_csv(StringIO(res.text.replace("=", "")), header=1,skiprows=6,skipfooter=7, engine='python')
   
    df1 = df1.dropna(how='all', axis=0).dropna(how='all', axis=1)
    df1=df1[df1['股票代號'].notna()]

    id=[]

    for i in range(len(df1)):
        id.append(str(df1['股票代號'][i]).replace(' ','')+' '+str(df1['股票名稱'][i]).replace(' ',''))
    #print(id)

    df1['stock_id']=id
    df1['date']=date

    df1=df1.set_index(['stock_id', 'date'])
    df1=df1.drop(columns=['股票代號', '股票名稱','註記'])
    df1=df1.rename(columns={"買進": "融資買進","賣出": "融資賣出","前日餘額": "融資前日餘額","今日餘額": "融資今日餘額","限額": "融資限額",
                            "買進.1": "融券買進","賣出.1": "融券賣出","前日餘額.1": "融券前日餘額","今日餘額.1": "融券今日餘額","限額.1": "融券限額"})
    df1 = df1.apply(lambda s: s.astype(str).str.replace(",", ""))
    df1 = df1.dropna(how='all', axis=0).dropna(how='all', axis=1)
    print(df1.sort_index(level='date', ascending=True).head(1).index.get_level_values('date'))
    return df1

In [2]:
#get last date of database
df = pd.read_pickle("history/tables/margin_trading.pkl")
df.index.get_level_values('date').drop_duplicates(keep = 'first') .sort_values()


DatetimeIndex(['2001-01-02', '2001-01-03', '2001-01-04', '2001-01-05',
               '2001-01-08', '2001-01-09', '2001-01-10', '2001-01-11',
               '2001-01-12', '2001-01-15',
               ...
               '2020-06-01', '2020-06-02', '2020-06-03', '2020-06-04',
               '2020-06-05', '2020-06-08', '2020-06-09', '2020-06-10',
               '2020-06-11', '2020-06-12'],
              dtype='datetime64[ns]', name='date', length=4803, freq=None)

In [10]:
df_combined=pd.DataFrame()
days=pd.date_range(start="2020-05-29",end="2020-06-12")
for date in days:
    df_tmp=MarginTrading(date)
    df_combined=df_combined.append(df_tmp)
    time.sleep(7)
#print(df_combined[df_combined.index.get_level_values('stock_id')=="1101 台泥"].head())

DatetimeIndex(['2020-05-29'], dtype='datetime64[ns]', name='date', freq=None)
holiday
holiday
DatetimeIndex(['2020-06-01'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2020-06-02'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2020-06-03'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2020-06-04'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2020-06-05'], dtype='datetime64[ns]', name='date', freq=None)
holiday
holiday
DatetimeIndex(['2020-06-08'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2020-06-09'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2020-06-10'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2020-06-11'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2020-06-12'], dtype='datetime64[ns]', name='date', freq=None)


In [ ]:
df_combined[df_combined.index.get_level_values('stock_id')=="1101 台泥"]
df_combined.index.get_level_values('date').drop_duplicates(keep = 'first')

In [13]:
#df = pd.read_pickle("history/tables/margin_trading.pkl")
df=df.append(df_combined)
df=df.sort_index()

df=df.drop_duplicates(keep = 'first', inplace = True) 
print(df[df.index.get_level_values('stock_id')=="1101 台泥"].head())
print(df[df.index.get_level_values('stock_id')=="1101 台泥"].tail())
to_pickle(df, 'margin_trading')

                     融資買進  融資賣出 現金償還 融資前日餘額 融資今日餘額    融資限額 融券買進 融券賣出 現券償還  \
stock_id date                                                               
1101 台泥  2001-01-02  1666  2096   33  73114  72651  593055  118   51    0   
         2001-01-03  2224  2697   22  72651  72156  593055  879  146    0   
         2001-01-04  4312  3665  132  72027  72542  593055  101  288    0   
         2001-01-05  3705  1863   56  72542  74328  593055  279    0    3   
         2001-01-08  3275  1977  375  74354  75277  593055   14    6    6   

                    融券前日餘額 融券今日餘額    融券限額  資券互抵  
stock_id date                                    
1101 台泥  2001-01-02   1449   1382  593055  1438  
         2001-01-03   1382    649  593055  2344  
         2001-01-04    644    831  593055  3618  
         2001-01-05    831    549  593055    11  
         2001-01-08    549    535  593055   560  
                    融資買進 融資賣出 現金償還 融資前日餘額 融資今日餘額     融資限額 融券買進 融券賣出 現券償還  \
stock_id date                     

In [9]:
#df[df.index.get_level_values('stock_id')=="1101 台泥"]
df.sort_index(level='date', ascending=True).index.get_level_values('date')

DatetimeIndex(['2001-01-02', '2001-01-02', '2001-01-02', '2001-01-02',
               '2001-01-02', '2001-01-02', '2001-01-02', '2001-01-02',
               '2001-01-02', '2001-01-02',
               ...
               '2020-06-12', '2020-06-12', '2020-06-12', '2020-06-12',
               '2020-06-12', '2020-06-12', '2020-06-12', '2020-06-12',
               '2020-06-12', '2020-06-12'],
              dtype='datetime64[ns]', name='date', length=3364167, freq=None)